In [ ]:
# Adapted from https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image

class FibreDataset(torch.utils.data.Dataset):
    def __init__(self, root, stage, transforms):
        self.root = root
        self.transforms = transforms
        self.stage = stage
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, 'Prepared', self.stage, 'images'))))
        self.masks = list(sorted(os.listdir(os.path.join(root, 'Prepared', self.stage, 'masks'))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, 'Prepared', self.stage, 'images', self.imgs[idx])
        mask_path = os.path.join(self.root, 'Prepared', self.stage, 'masks', self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the Image into a numpy array
        mask = np.array(mask)

        # Create an empty list to store the binary masks
        
        # Get the contours of the image - to find the grid polygons
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        masks = []
        # Iterate over each contour and plot it
        for contour in contours:
            # Create a new binary mask for the current contour
            contour_mask = np.zeros_like(mask, dtype=np.uint8)     
               # Draw the contour on the binary mask
            cv2.drawContours(contour_mask, [contour], 0, (255), thickness=cv2.FILLED)
    # Append the contour mask to the masks list
            masks.append(contour_mask)       
            

        num_objs = len(contours)
        boxes = torch.zeros([num_objs,4], dtype=torch.float32)

        for i in range(num_objs):
            x,y,w,h = cv2.boundingRect(contours[i])
            boxes[i] = torch.tensor([x, y, x+w, y+h])

        # Convert the masks list to a single NumPy array
        masks = np.array(masks)
        
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)    

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
   # train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

current_directory = '/content/drive/MyDrive/Colab Notebooks/FibreAnalysis'

# our dataset has two classes only - background and person
num_classes = 2
# use our dataset and defined transformations
dataset = FibreDataset(current_directory, 'Train',  get_transform(train=True))
dataset_test = FibreDataset(current_directory, 'Test', get_transform(train=False))

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=2,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)


num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)

# move model to the right device
model.to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)  # https://towardsdatascience.com/train-mask-rcnn-net-for-object-detection-in-60-lines-of-code-9b6bbff292c3

num_epochs = 1
print('Starting Training')
for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
    # optimizer.step() this is done in train_one_epoch method so not needed here #
    #  test whetherscheduler makes any difference
    #lr_scheduler.step() Let the optimiser 
        # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")    

Predictions 

In [ ]:
# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])
    
# show prediction
prediction    

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

In [ ]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())